# Mailchimp API Authentication and Setup

This section handles the complete authentication and setup process required to interact with the Mailchimp API.

**The process is as follows:**
1.  **Import Libraries:** Load necessary Python libraries (`requests`, `json`, `google.cloud.secretmanager`).
2.  **Set Configuration:** Define constants for the GCP Project ID and the name of the Mailchimp token secret in Secret Manager.
3.  **Retrieve Credentials:**
    * Connect to Google Cloud Secret Manager.
    * Fetch the stored secret, which contains a JSON object with the `access_token` and `dc` (server prefix).
4.  **Prepare for API Calls:**
    * Parse the JSON credentials.
    * Dynamically construct the `API_BASE_URL` using the server prefix.
    * Create the `headers` dictionary with the Bearer token for authorization.
5.  **Connection Test:**
    * Call the `/ping` endpoint to verify that the credentials and connection are valid. A successful test returns a `200` status code.

In [4]:
import sys
import requests
import json
from google.cloud import secretmanager # Direct import is fine for notebooks

print("Libraries imported successfully.")

Libraries imported successfully.


In [5]:
# --- Mailchimp Configuration ---
GCP_PROJECT_ID = "mis581-capstone-data"

# The ID of the secret in Secret Manager containing the Mailchimp tokens
# This comes from your successful OAuth flow.
MC_TOKEN_SECRET_NAME = "peer2_mailchimp_token"

print("Mailchimp configuration variables are set.")

Mailchimp configuration variables are set.


In [6]:
# Initialize clients
secret_client = secretmanager.SecretManagerServiceClient()

try:
    # 1. Fetch the secret's payload
    secret_name = f"projects/{GCP_PROJECT_ID}/secrets/{MC_TOKEN_SECRET_NAME}/versions/latest"
    response = secret_client.access_secret_version(request={"name": secret_name})
    payload = response.payload.data.decode("UTF-8")
    
    # 2. Parse the JSON string from the secret
    credentials = json.loads(payload)
    access_token = credentials.get("access_token")
    server_prefix = credentials.get("dc") # 'dc' is the data center/server prefix

    if not access_token or not server_prefix:
        raise ValueError("'access_token' or 'dc' not found in the secret payload.")

    # 3. Dynamically build the API URL and headers
    API_BASE_URL = f"https://{server_prefix}.api.mailchimp.com/3.0"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    print("Successfully retrieved Mailchimp credentials and prepared API details.")
    print(f"API Base URL configured for server prefix: {server_prefix}")

except Exception as e:
    print(f"An error occurred during setup: {e}")
    # Set to None to prevent subsequent cells from running with bad data
    API_BASE_URL = None

Successfully retrieved Mailchimp credentials and prepared API details.
API Base URL configured for server prefix: us18


In [7]:
if API_BASE_URL:
    # Use the /ping endpoint for a simple health check
    test_endpoint = "/ping"
    full_url = API_BASE_URL + test_endpoint
    
    print(f"Making a test API call to: {full_url}")
    
    try:
        response = requests.get(full_url, headers=headers, timeout=10)
        print("API call sent successfully. Check the next cell for the response.")
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
        response = None
else:
    print("Cannot make API call, setup in the previous cell failed.")
    response = None

Making a test API call to: https://us18.api.mailchimp.com/3.0/ping
API call sent successfully. Check the next cell for the response.


In [7]:
if response:
    print(f"HTTP Status Code: {response.status_code}")
    
    if response.status_code == 200:
        print("\nSUCCESS! The API call to Mailchimp worked.")
        print("Here is the response from the server:")
        # Pretty-print the JSON response
        print(json.dumps(response.json(), indent=2))
    else:
        print("\nERROR: The API call failed.")
        print("Here is the error response:")
        # Print the raw text of the error
        print(response.text)
else:
    print("Response object is None. The API call likely did not complete.")

AttributeError: Unknown field for AccessSecretVersionResponse: status_code

# Data Extraction from Mailchimp API

Now that authentication is successful, this section focuses on the primary goal: extracting marketing data from Mailchimp endpoints and preparing it for loading into Google BigQuery.

### Part 1: Fetching List Members

The first step is to download all contacts (list members) from the primary audience.

**The process is as follows:**
1.  **Define Extraction Function:** Create a Python function (`fetch_all_mailchimp_members`) that takes the API credentials and a `list_id` as input.
2.  **Handle Pagination:** Inside the function, implement a `while` loop to repeatedly call the API, using the `offset` and `count` parameters to navigate through all pages of results until no more members are returned.
3.  **Execute Fetch:** Call the function to retrieve the complete list of all members.
4.  **Structure for BigQuery:**
    * Loop through the raw list of members returned from the API.
    * For each member, create a dictionary with two keys:
        * `raw_data`: A JSON string of the complete, unaltered member record.
        * [cite_start]`tenant_id`: A string identifier for the data source (e.g., 'peer2') to support multi-tenancy[cite: 152, 161, 167, 174].
5.  [cite_start]**Load to BigQuery:** Pass the final list of structured records to a loading function that writes them to the appropriate raw data table (e.g., `peer2_mailchimp_members_raw`)[cite: 128, 154, 168].

In [8]:
import json
import requests
from google.cloud import secretmanager

# ===================================================================
#                      CONFIGURATION
# ===================================================================
GCP_PROJECT_ID = "mis581-capstone-data"
MC_TOKEN_SECRET_NAME = "peer2_mailchimp_token"

print("--- Step 1: Authenticating with Google and Mailchimp ---")

# ===================================================================
#                      AUTHENTICATION LOGIC
# ===================================================================
try:
    # Initialize Google Cloud client
    secret_client = secretmanager.SecretManagerServiceClient()
    
    # Use a specific variable name for the secret manager response
    secret_name = f"projects/{GCP_PROJECT_ID}/secrets/{MC_TOKEN_SECRET_NAME}/versions/latest"
    secret_response = secret_client.access_secret_version(request={"name": secret_name})
    
    # Decode the payload from the specific secret_response object
    payload = secret_response.payload.data.decode("UTF-8")
    
    credentials = json.loads(payload)
    access_token = credentials.get("access_token")
    server_prefix = credentials.get("dc")

    if not access_token or not server_prefix:
        raise ValueError("'access_token' or 'dc' not found in the secret payload.")

    API_BASE_URL = f"https://{server_prefix}.api.mailchimp.com/3.0"
    headers = { "Authorization": f"Bearer {access_token}" }
    
    print("Authentication successful. Prepared API details.")
    print(f"API Base URL configured for server prefix: {server_prefix}")

except Exception as e:
    API_BASE_URL = None
    headers = None
    print(f"!!! An error occurred during the AUTHENTICATION phase: {e}")

# ===================================================================
#                 FETCH ALL LISTS (AUDIENCES)
# ===================================================================
# This part only runs if authentication was successful
if API_BASE_URL and headers:
    print("\n--- Step 2: Fetching all Mailchimp Lists (Audiences) ---")
    
    endpoint = "/lists"
    full_url = API_BASE_URL + endpoint

    try:
        # Use a specific variable name for the Mailchimp API response
        mc_response = requests.get(full_url, headers=headers, timeout=15)
        mc_response.raise_for_status() # Check for HTTP errors
        
        mailchimp_lists_data = mc_response.json()
        print("\nSUCCESS: API call was successful.")
        
        lists_found = mailchimp_lists_data.get('lists', [])
        if lists_found:
            print(f"Found {len(lists_found)} list(s) in the account:")
            for list_item in lists_found:
                print(f"  - List Name: \"{list_item.get('name')}\", ID: {list_item.get('id')}")
        else:
            print("No lists (audiences) were found in this Mailchimp account.")

    except requests.exceptions.HTTPError as e:
        print(f"\nERROR: The API call to Mailchimp failed with status code {e.response.status_code}.")
        print(f"Response Body: {e.response.text}")
    except Exception as e:
        print(f"An unexpected error occurred while fetching lists: {e}")

--- Step 1: Authenticating with Google and Mailchimp ---
Authentication successful. Prepared API details.
API Base URL configured for server prefix: us18

--- Step 2: Fetching all Mailchimp Lists (Audiences) ---

SUCCESS: API call was successful.
Found 10 list(s) in the account:
  - List Name: "2023-2024 Free Trials Musikgarten, Music Immersion, Pre-Instruments", ID: 0178eb30f5
  - List Name: "Current Private Lessons Students 2024-2025", ID: 1477246008
  - List Name: "Summer Camps 2024", ID: 3b93b1512e
  - List Name: "Musikgarten and Music Immersion Enrollment 2019-2023", ID: 555a29bd42
  - List Name: "Private Lessons- All Time", ID: 5c021fd130
  - List Name: "Current Private Lessons Students 2023-2024", ID: 6fa6cd3238
  - List Name: "Spring 2023 Students", ID: 8bbe106a9f
  - List Name: "Winter 2024 Students", ID: 98395f51cf
  - List Name: "Free Trials December 2022-February 2023", ID: 9d6561c536
  - List Name: "Free Trials - Musikgarten, Music Immersion, and Pre-Instruments March - Au

In [11]:
import json
import requests
import time
from google.cloud import secretmanager

# ===================================================================
#                      1. CONFIGURATION
# ===================================================================
GCP_PROJECT_ID = "mis581-capstone-data"
MC_TOKEN_SECRET_NAME = "peer2_mailchimp_token"
print("Configuration set.")

# ===================================================================
#                      2. AUTHENTICATION
# ===================================================================
print("\n--- Authenticating with Google and Mailchimp ---")
try:
    # THIS LINE IS NOW CORRECTED
    secret_client = secretmanager.SecretManagerServiceClient()
    
    secret_name = f"projects/{GCP_PROJECT_ID}/secrets/{MC_TOKEN_SECRET_NAME}/versions/latest"
    secret_response = secret_client.access_secret_version(request={"name": secret_name})
    payload = secret_response.payload.data.decode("UTF-8")
    
    credentials = json.loads(payload)
    mc_access_token = credentials.get("access_token")
    mc_server_prefix = credentials.get("dc")

    if not mc_access_token or not mc_server_prefix:
        raise ValueError("'access_token' or 'dc' not found in the secret payload.")

    API_BASE_URL = f"https://{mc_server_prefix}.api.mailchimp.com/3.0"
    headers = { "Authorization": f"Bearer {mc_access_token}" }
    
    print("Authentication successful.")
except Exception as e:
    API_BASE_URL = None
    print(f"!!! An error occurred during the AUTHENTICATION phase: {e}")

# ===================================================================
#           3. FUNCTION DEFINITION FOR FETCHING MEMBERS
# ===================================================================
def fetch_all_mailchimp_members(token, server_prefix, list_id):
    """Fetches all members from a Mailchimp list, handling pagination."""
    all_members = []
    page_size = 200
    offset = 0
    
    api_url = f"https://{server_prefix}.api.mailchimp.com/3.0/lists/{list_id}/members"
    auth_headers = {"Authorization": f"Bearer {token}"}

    while True:
        params = {"count": page_size, "offset": offset}
        try:
            response = requests.get(api_url, headers=auth_headers, params=params)
            response.raise_for_status()
            data = response.json()
            
            members = data.get('members', [])
            if not members: break
            
            all_members.extend(members)
            print(f"  Fetched {len(members)} members, total for this list so far: {len(all_members)}")
            
            offset += page_size
            time.sleep(0.5)
        except requests.exceptions.HTTPError as e:
            print(f"  !!! HTTP Error fetching members: {e}")
            break
            
    return all_members

# ===================================================================
#                   4. MAIN EXECUTION LOGIC
# ===================================================================
# This logic only runs if authentication was successful
if API_BASE_URL:
    print("\n--- Fetching all Mailchimp Lists (Audiences) ---")
    try:
        mc_response = requests.get(f"{API_BASE_URL}/lists", headers=headers, timeout=15)
        mc_response.raise_for_status()
        lists_found = mc_response.json().get('lists', [])
        
        if lists_found:
            print(f"Found {len(lists_found)} list(s). Now fetching members from each.")
            
            # This dictionary will hold all the final data
            all_data = {}
            
            for list_item in lists_found:
                list_id = list_item.get('id')
                list_name = list_item.get('name')
                print(f"\nProcessing List: \"{list_name}\" (ID: {list_id})")
                
                # Fetch all members for the current list_id
                members_of_list = fetch_all_mailchimp_members(mc_access_token, mc_server_prefix, list_id)
                
                all_data[list_id] = {
                    "name": list_name,
                    "members": members_of_list,
                    "member_count": len(members_of_list)
                }
                print(f"-> Finished \"{list_name}\". Found {len(members_of_list)} members.")
            
            print("\n\n--- ALL DATA EXTRACTION COMPLETE ---")
            total_members = sum(d['member_count'] for d in all_data.values())
            print(f"Successfully fetched a grand total of {total_members} members across {len(all_data)} lists.")
            # The 'all_data' variable now holds all your contact information.
            
        else:
            print("No lists (audiences) were found in this Mailchimp account.")
            
    except Exception as e:
        print(f"An unexpected error occurred during the main execution: {e}")

Configuration set.

--- Authenticating with Google and Mailchimp ---
Authentication successful.

--- Fetching all Mailchimp Lists (Audiences) ---
Found 10 list(s). Now fetching members from each.

Processing List: "2023-2024 Free Trials Musikgarten, Music Immersion, Pre-Instruments" (ID: 0178eb30f5)
  Fetched 105 members, total for this list so far: 105
-> Finished "2023-2024 Free Trials Musikgarten, Music Immersion, Pre-Instruments". Found 105 members.

Processing List: "Current Private Lessons Students 2024-2025" (ID: 1477246008)
  Fetched 183 members, total for this list so far: 183
-> Finished "Current Private Lessons Students 2024-2025". Found 183 members.

Processing List: "Summer Camps 2024" (ID: 3b93b1512e)
  Fetched 200 members, total for this list so far: 200
  Fetched 200 members, total for this list so far: 400
  Fetched 62 members, total for this list so far: 462
-> Finished "Summer Camps 2024". Found 462 members.

Processing List: "Musikgarten and Music Immersion Enrollme

In [13]:
import json
from google.cloud import bigquery

# ===================================================================
#                      CONFIGURATION
# ===================================================================
TENANT_ID = "peer2"
BIGQUERY_PROJECT_ID = "mis581-capstone-data"
BIGQUERY_TABLE_ID = f"{BIGQUERY_PROJECT_ID}.raw_data.peer2_members_raw"
print("Configuration for BigQuery load is set.")

# ===================================================================
#                BIGQUERY LOADING UTILITY FUNCTION
# ===================================================================
def load_data_to_bigquery(table_id, data_list):
    """Loads a list of dictionary records into the specified BigQuery table."""
    if not data_list:
        print("No data to load.")
        return

    bq_client = bigquery.Client(project=BIGQUERY_PROJECT_ID)
    
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("raw_data", "STRING"),
            bigquery.SchemaField("tenant_id", "STRING"),
        ],
        # If your raw_data column is JSON type in BQ, use WRITE_TRUNCATE or WRITE_APPEND
        write_disposition="WRITE_TRUNCATE", # This will overwrite the table
    )
    
    print(f"Initiating load job for {len(data_list)} records into {table_id}...")
    load_job = bq_client.load_table_from_json(data_list, table_id, job_config=job_config)
    load_job.result() # Wait for the job to complete
    
    print(f"SUCCESS: BigQuery load job finished for table {table_id}.")
    print(f"Loaded {load_job.output_rows} rows.")

# ===================================================================
#                         MAIN EXECUTION
# ===================================================================
# Check if the 'all_data' variable from the previous cell exists.
if 'all_data' in globals() and all_data:
    # Step 1: Flatten the data from the dictionary into a single list of members.
    all_members_flat_list = []
    for list_id in all_data:
        all_members_flat_list.extend(all_data[list_id]['members'])
    
    # THIS IS THE CORRECTED LOGIC:
    # First, we confirm we have a list, then we use ITS length.
    if all_members_flat_list:
        print(f"\n--- Preparing {len(all_members_flat_list)} records for BigQuery load ---")

        # [cite_start]Step 2: Structure the data for BigQuery loading. [cite: 61, 78, 99, 123]
        records_to_load = []
        for member in all_members_flat_list:
            record = {
                "raw_data": json.dumps(member),
                "tenant_id": TENANT_ID
            }
            records_to_load.append(record)
        
        # [cite_start]Step 3: Load the prepared data into BigQuery. [cite: 5, 117, 159, 164]
        try:
            load_data_to_bigquery(BIGQUERY_TABLE_ID, records_to_load)
        except Exception as e:
            print(f"!!! An error occurred during the BigQuery load job: {e}")
    else:
        print("--- Flattened data list is empty. Nothing to load. ---")
else:
    print("!!! ERROR: Could not find the 'all_data' variable. Please re-run the previous extraction cell first. !!!")

Configuration for BigQuery load is set.

--- Preparing 4065 records for BigQuery load ---
Initiating load job for 4065 records into mis581-capstone-data.raw_data.peer2_members_raw...
SUCCESS: BigQuery load job finished for table mis581-capstone-data.raw_data.peer2_members_raw.
Loaded 4065 rows.


In [14]:
import json
import requests
import time
from google.cloud import secretmanager
from google.cloud import bigquery

# ===================================================================
#                      1. CONFIGURATION
# ===================================================================
GCP_PROJECT_ID = "mis581-capstone-data"
MC_TOKEN_SECRET_NAME = "peer2_mailchimp_token"
TENANT_ID = "peer2"
BIGQUERY_CAMPAIGNS_TABLE_ID = f"{GCP_PROJECT_ID}.raw_data.peer2_campaigns_raw"
print("Configuration set for Campaign extraction.")

# ===================================================================
#                      2. AUTHENTICATION
# ===================================================================
print("\n--- Authenticating with Google and Mailchimp ---")
try:
    secret_client = secretmanager.SecretManagerServiceClient()
    secret_name = f"projects/{GCP_PROJECT_ID}/secrets/{MC_TOKEN_SECRET_NAME}/versions/latest"
    secret_response = secret_client.access_secret_version(request={"name": secret_name})
    payload = secret_response.payload.data.decode("UTF-8")
    
    credentials = json.loads(payload)
    mc_access_token = credentials.get("access_token")
    mc_server_prefix = credentials.get("dc")

    if not mc_access_token or not mc_server_prefix:
        raise ValueError("'access_token' or 'dc' not found in the secret payload.")

    API_BASE_URL = f"https://{mc_server_prefix}.api.mailchimp.com/3.0"
    print("Authentication successful.")
except Exception as e:
    API_BASE_URL = None
    print(f"!!! An error occurred during the AUTHENTICATION phase: {e}")

# ===================================================================
#           3. UTILITY FUNCTIONS (FETCH & LOAD)
# ===================================================================
def fetch_all_mailchimp_campaigns(token, server_prefix):
    """Fetches all campaigns from a Mailchimp account, handling pagination."""
    all_campaigns = []
    page_size = 100
    offset = 0
    
    api_url = f"https://{server_prefix}.api.mailchimp.com/3.0/campaigns"
    auth_headers = {"Authorization": f"Bearer {token}"}

    while True:
        params = {"count": page_size, "offset": offset}
        try:
            response = requests.get(api_url, headers=auth_headers, params=params)
            response.raise_for_status()
            data = response.json()
            
            campaigns = data.get('campaigns', [])
            if not campaigns: break
            
            all_campaigns.extend(campaigns)
            print(f"  Fetched {len(campaigns)} campaigns, total so far: {len(all_campaigns)}")
            
            offset += page_size
            time.sleep(0.5)
        except requests.exceptions.HTTPError as e:
            print(f"  !!! HTTP Error fetching campaigns: {e}")
            break
            
    return all_campaigns

def load_data_to_bigquery(table_id, data_list):
    """Loads a list of dictionary records into the specified BigQuery table."""
    if not data_list:
        print("No data to load.")
        return

    bq_client = bigquery.Client(project=GCP_PROJECT_ID)
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("raw_data", "STRING"),
            bigquery.SchemaField("tenant_id", "STRING"),
        ],
        write_disposition="WRITE_TRUNCATE",
    )
    
    print(f"\nInitiating load job for {len(data_list)} records into {table_id}...")
    load_job = bq_client.load_table_from_json(data_list, table_id, job_config=job_config)
    load_job.result()
    
    print(f"SUCCESS: BigQuery load job finished. Loaded {load_job.output_rows} rows.")

# ===================================================================
#                   4. MAIN EXECUTION LOGIC
# ===================================================================
if API_BASE_URL:
    print("\n--- Starting Mailchimp Campaign Data Extraction ---")
    
    # Step 1: Fetch all campaigns
    all_campaigns = fetch_all_mailchimp_campaigns(mc_access_token, mc_server_prefix)
    
    if all_campaigns:
        print(f"\n--- Extraction Complete: Found {len(all_campaigns)} total campaigns. ---")
        
        # Step 2: Structure data for BigQuery
        records_to_load = []
        for campaign in all_campaigns:
            record = {
                "raw_data": json.dumps(campaign),
                "tenant_id": TENANT_ID
            }
            records_to_load.append(record)

        # Step 3: Load data into BigQuery
        try:
            load_data_to_bigquery(BIGQUERY_CAMPAIGNS_TABLE_ID, records_to_load)
        except Exception as e:
            print(f"!!! An error occurred during the BigQuery load job: {e}")
    else:
        print("No campaigns found in the account.")
else:
    print("Execution halted due to authentication failure.")

Configuration set for Campaign extraction.

--- Authenticating with Google and Mailchimp ---
Authentication successful.

--- Starting Mailchimp Campaign Data Extraction ---
  Fetched 100 campaigns, total so far: 100
  Fetched 100 campaigns, total so far: 200
  Fetched 100 campaigns, total so far: 300
  Fetched 100 campaigns, total so far: 400
  Fetched 100 campaigns, total so far: 500
  Fetched 100 campaigns, total so far: 600
  Fetched 100 campaigns, total so far: 700
  Fetched 100 campaigns, total so far: 800
  Fetched 100 campaigns, total so far: 900
  Fetched 100 campaigns, total so far: 1000
  Fetched 100 campaigns, total so far: 1100
  Fetched 100 campaigns, total so far: 1200
  Fetched 100 campaigns, total so far: 1300
  Fetched 100 campaigns, total so far: 1400
  Fetched 100 campaigns, total so far: 1500
  Fetched 100 campaigns, total so far: 1600
  Fetched 100 campaigns, total so far: 1700
  Fetched 100 campaigns, total so far: 1800
  Fetched 100 campaigns, total so far: 1900


In [15]:
import json
import requests
import time
from google.cloud import secretmanager
from google.cloud import bigquery

# ===================================================================
#                      1. CONFIGURATION
# ===================================================================
GCP_PROJECT_ID = "mis581-capstone-data"
MC_TOKEN_SECRET_NAME = "peer2_mailchimp_token"
TENANT_ID = "peer2"
BIGQUERY_REPORTS_TABLE_ID = f"{GCP_PROJECT_ID}.raw_data.peer2_reports_raw"
print("Configuration set for Campaign Report extraction.")

# ===================================================================
#                      2. AUTHENTICATION
# ===================================================================
print("\n--- Authenticating with Google and Mailchimp ---")
try:
    secret_client = secretmanager.SecretManagerServiceClient()
    secret_name = f"projects/{GCP_PROJECT_ID}/secrets/{MC_TOKEN_SECRET_NAME}/versions/latest"
    secret_response = secret_client.access_secret_version(request={"name": secret_name})
    payload = secret_response.payload.data.decode("UTF-8")
    
    credentials = json.loads(payload)
    mc_access_token = credentials.get("access_token")
    mc_server_prefix = credentials.get("dc")

    if not mc_access_token or not mc_server_prefix:
        raise ValueError("'access_token' or 'dc' not found in the secret payload.")

    API_BASE_URL = f"https://{mc_server_prefix}.api.mailchimp.com/3.0"
    print("Authentication successful.")
except Exception as e:
    API_BASE_URL = None
    print(f"!!! An error occurred during the AUTHENTICATION phase: {e}")

# ===================================================================
#           3. UTILITY FUNCTIONS (FETCH & LOAD)
# ===================================================================
def fetch_all_mailchimp_campaigns(token, server_prefix):
    """Fetches all campaigns from a Mailchimp account, handling pagination."""
    all_campaigns = []
    page_size = 100
    offset = 0
    api_url = f"https://{server_prefix}.api.mailchimp.com/3.0/campaigns"
    auth_headers = {"Authorization": f"Bearer {token}"}
    while True:
        params = {"count": page_size, "offset": offset, "fields": "campaigns.id"} # Only need the ID
        try:
            response = requests.get(api_url, headers=auth_headers, params=params)
            response.raise_for_status()
            data = response.json()
            campaigns = data.get('campaigns', [])
            if not campaigns: break
            all_campaigns.extend(campaigns)
            offset += page_size
        except requests.exceptions.HTTPError as e:
            print(f"  !!! HTTP Error fetching campaigns: {e}")
            break
    return all_campaigns

def fetch_campaign_reports(token, server_prefix, campaigns_list):
    """Loops through campaigns and fetches the report for each one."""
    all_reports = []
    auth_headers = {"Authorization": f"Bearer {token}"}
    
    for i, campaign in enumerate(campaigns_list):
        campaign_id = campaign.get('id')
        if not campaign_id: continue

        print(f"  Fetching report {i+1} of {len(campaigns_list)} for campaign ID: {campaign_id}")
        report_url = f"https://{server_prefix}.api.mailchimp.com/3.0/reports/{campaign_id}"
        
        try:
            response = requests.get(report_url, headers=auth_headers)
            response.raise_for_status()
            report_data = response.json()
            
            # Add the campaign_id to the report data for easier joins later [cite: 110]
            report_data['campaign_id'] = campaign_id
            all_reports.append(report_data)
            time.sleep(0.5) # Be courteous to the API
        except requests.exceptions.HTTPError as e:
            # A 404 error is common for campaigns that have no report data (e.g., drafts)
            if e.response.status_code == 404:
                print(f"    -> No report found for campaign {campaign_id} (Status 404). Skipping.")
            else:
                print(f"    -> HTTP Error for campaign {campaign_id}: {e}")
                
    return all_reports

def load_data_to_bigquery(table_id, data_list):
    """Loads a list of dictionary records into the specified BigQuery table."""
    if not data_list:
        print("No data to load.")
        return

    bq_client = bigquery.Client(project=GCP_PROJECT_ID)
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("raw_data", "STRING"),
            bigquery.SchemaField("tenant_id", "STRING"),
        ],
        write_disposition="WRITE_TRUNCATE",
    )
    
    print(f"\nInitiating load job for {len(data_list)} records into {table_id}...")
    load_job = bq_client.load_table_from_json(data_list, table_id, job_config=job_config)
    load_job.result()
    
    print(f"SUCCESS: BigQuery load job finished. Loaded {load_job.output_rows} rows.")

# ===================================================================
#                   4. MAIN EXECUTION LOGIC
# ===================================================================
if API_BASE_URL:
    print("\n--- Starting Mailchimp Campaign Report Data Extraction ---")
    
    # Step 1: Get a fresh list of all campaign IDs. We only need the ID field.
    print("First, getting a list of all campaign IDs...")
    all_campaigns = fetch_all_mailchimp_campaigns(mc_access_token, mc_server_prefix)
    
    if all_campaigns:
        print(f"Found {len(all_campaigns)} campaigns. Now fetching a report for each.")
        
        # Step 2: Fetch the report for each campaign
        all_reports = fetch_campaign_reports(mc_access_token, mc_server_prefix, all_campaigns)
        
        if all_reports:
            print(f"\n--- Extraction Complete: Found {len(all_reports)} total reports. ---")

            # Step 3: Structure data for BigQuery
            records_to_load = []
            for report in all_reports:
                record = {
                    "raw_data": json.dumps(report),
                    "tenant_id": TENANT_ID
                }
                records_to_load.append(record)

            # Step 4: Load data into BigQuery [cite: 76, 111]
            try:
                load_data_to_bigquery(BIGQUERY_REPORTS_TABLE_ID, records_to_load)
            except Exception as e:
                print(f"!!! An error occurred during the BigQuery load job: {e}")
        else:
            print("No reports were successfully fetched.")
    else:
        print("No campaigns found, cannot fetch reports.")
else:
    print("Execution halted due to authentication failure.")

Configuration set for Campaign Report extraction.

--- Authenticating with Google and Mailchimp ---
Authentication successful.

--- Starting Mailchimp Campaign Report Data Extraction ---
First, getting a list of all campaign IDs...
Found 2160 campaigns. Now fetching a report for each.
  Fetching report 1 of 2160 for campaign ID: b2fcd2fe98
  Fetching report 2 of 2160 for campaign ID: 972fb02471
  Fetching report 3 of 2160 for campaign ID: d7ffb8c76c
  Fetching report 4 of 2160 for campaign ID: 3daeaccb4e
  Fetching report 5 of 2160 for campaign ID: 4a46fdddaa
  Fetching report 6 of 2160 for campaign ID: 4854dd261e
  Fetching report 7 of 2160 for campaign ID: 38937e2ccb
  Fetching report 8 of 2160 for campaign ID: 655a59525d
  Fetching report 9 of 2160 for campaign ID: 1fe1e3df2b
  Fetching report 10 of 2160 for campaign ID: 4d8be3fbe7
  Fetching report 11 of 2160 for campaign ID: b5971a5265
  Fetching report 12 of 2160 for campaign ID: b860727e87
  Fetching report 13 of 2160 for campai